In [80]:

import nltk
import csv
import os
from nltk.tokenize import sent_tokenize,word_tokenize
import re
import numpy as np

In [135]:
def sentence_section(f):
    sents_list=[]
    section={}
    for para in f:
        p=sent_tokenize(para)
        for sent in p:
            if sent != []:
                if  len(sent)<30:
                     section[len(sents_list)]=sent
                else:
                     sents_list.append(sent)
   

    return(section, sents_list)


In [4]:
keywords={1:["eligibility", "inclusion", "exclusion", "criteria"],
2:["random", "alternate", "allocate"],
3:["allocation", "concealed","concealment"],
4:["baseline", "characteristic", "age", "sex", "gender", "demographic"],
5:["mask","masking","blinding","blind"],
6:["mask","masking","blinding","blind"],
7:["mask","masking","blinding","blind"],
8:["dropout",  "attrition", "withdraw", "withdrew", "complete(d)","complete"],
9:["intention-to-treat", "intention"],
10:["inter-group", "between-group", "anova", "t-test", "analysis","variance"],
11:["mean", "standard", "deviation", "sd", "error", "sem", "se", "variance"]}

In [73]:

headers=["PubMedID","#sentences","#Q1(K1/K2)","#Q2(K1/K2)","#Q3(K1/K2)","#Q4(K1/K2)",
         "#Q5(K1/K2)","#Q8(K1/K2)","#Q9(K1/K2)","#Q10(K1/K2)","#Q11(K1/K2)"]
head=["Sentence-ID","Sentence-String","Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9", "Q10","Q11","Section"]

In [136]:
filelist=[]
for filename in os.listdir('/Users/veronica/Documents/research/pmarticle'):
    filelist.append(filename)

In [23]:

with open('a_statistic.csv','w',newline='') as f:
    writer=csv.writer(f)
    csv.writer(f).writerow(headers)
    for name in filelist[2:-2]:
        csvrow=[name]
        f=open(name,'r')
        section, sents=sentence_section(f)
        length=len(sents)
        csvrow.append(length)
        for i in {1,2,3,4,5,8,9,10,11}:
            one,two=find_keywords(sents,length,keywords[i])
            csvrow.append(str(one)+'/'+str(two))
        writer.writerow(csvrow)
        f.close() 

In [8]:
for file in filelist[2:-2]:
    with open('/Users/veronica/Documents/research/sentences/{}.csv'.format(file),'w',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(head)
        f1=open(file,'r')
        section, sents=sentence_section(f1)
        f1.close()
        for i in range(len(sents)):
            csvrow=[i,sents[i]]
            word_list=word_tokenize(sents[i].lower())
            for j in keywords.keys():
                flag=0
                for key in keywords[j]:
                    if key in word_list:
                        flag=1
                csvrow.append(flag)
            if i in section.keys():
                csvrow.append(section[i])
        
            writer.writerow(csvrow) 

In [138]:
import csv
with open("/Users/veronica/Documents/research/data.csv", "r", encoding = 'unicode_escape') as csvfile:
    reader = csv.reader(csvfile)
    rows= [row for row in reader]


In [70]:
for i in range(len(rows[0])):
    if rows[0][i]=='articleId':
        print(i)
for i in range(len(rows[0])):
    if rows[0][i]=='Q1 Comment':
        print(i)

14
23


In [147]:
import csv
with open("/Users/veronica/Documents/research/data.csv", "r", encoding = 'unicode_escape') as csvfile:
    reader = csv.reader(csvfile)
    rows= [row for row in reader]
    
head1=["Sentence-ID","Sentence-String","Q1","Q2","Q3","Q4","Q5","Q6","Q7","Q8","Q9", "Q10","Q11"]
question=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8','Q9','Q10','Q11']
for file in filelist[2:-2]:
    with open('/Users/veronica/Documents/research/standard/{}.csv'.format(file),'w',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(head1)
        f1=open(file,'r')
        section, sents=sentence_section(f1)
        f1.close()
        
        row=[]
        for i in range(len(rows)):
            if rows[i][14]==file:
                row.append(i)
        for i in range(11):
            question[i]=[]
            for j in row:
                question[i].append(rows[j][i*2+23])
                

        sec=''
        for i in range(len(sents)):
            csvrow=[i,sents[i]]
            
            if i in section.keys():
                sec=section[i]
            for j in range(11):
                flag=0
                if question[j]==['']:
                    csvrow.append("N")
                else:
                    for com in question[j]:
                        if len(com)<30:
                            if difflib.SequenceMatcher(None, com, sec).ratio()>0.5:
                                flag=1
                        else:
                            if difflib.SequenceMatcher(None, com, sents[i]).ratio()>0.5:
                                flag=1 
                
                    csvrow.append(flag)            
            writer.writerow(csvrow)            
            
                    
                
        
            
            
           

In [131]:
import difflib
print(difflib.SequenceMatcher(None, "abc", "abc").ratio())

1.0


In [196]:
path1 = "/Users/veronica/Documents/research/sentences"
path2 = "/Users/veronica/Documents/research/standard"
count=[0,0,0,0,0,0,0,0,0,0,0]
total=[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
filename=[]
for file in os.listdir(path1)[3:]:
    with open(os.path.join(path1, file), 'r',encoding = 'unicode_escape') as f1:
        reader1 = csv.reader(f1)
        rows1= [row for row in reader1]
    with open(os.path.join(path2, file), 'r',encoding = 'unicode_escape') as f2:
        reader2 = csv.reader(f2)
        rows2= [row for row in reader2]
    N=len(rows1)
    for i in range(2,13):
        if rows2[2][i]!='N':
            total[i-2]+=N
            for j in range(1,N):
                if rows2[j][i]!=rows1[j][i]:
                    count[i-2]+=1
                
proportion=[]     #mistake            
for i in range(11):
    proportion.append(count[i]/total[i])
print(proportion)

[0.053357395804957776, 0.04434743443162526, 0.04343850022862369, 0.1107666717072433, 0.036284777319369514, 0.034897405720718415, 0.03563990912533131, 0.055169235924932974, 0.04381443298969072, 0.08674417757043408, 0.10606653620352251]
